In [1]:
import sys
sys.path.append('./privatizers')
sys.path.append('./dataProviders')
sys.path.append('./adapters')

from sklearn import svm
from sklearn.datasets import fetch_covtype
from adapters import generalAdapter as adapter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dp = fetch_covtype()

X = dp.data
Y = dp.target

In [2]:
# ## Size adjustment
# desired_size = 1000
# min_test_size = 0.3
# test_size = 0.3

# total_size = len(X)

# if (desired_size < min_test_size * total_size):
#     test_size = 1-(desired_size / total_size);

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

# ## Dataset
# data_set = {
#     'samples': X_train,
#     'labels': y_train
# }

In [3]:
## Multiple Dataset sizes
desired_sizes = [10, 100, 1000, 10000]
data_sets = []

min_test_size = 0.2

total_size = len(X)

for desired_size in desired_sizes:
    test_size = 0.2
    if (desired_size < (1-min_test_size) * total_size):
        test_size = 1-(desired_size / total_size);

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

    ## Dataset
    single_data_set = {
        'samples': X_train,
        'labels': y_train
    }
    data_sets.append(single_data_set)

In [4]:
## SVM without privatization
for data_set in data_sets:
    clf = svm.SVC(gamma='scale')
    clf.fit(data_set['samples'], data_set['labels'])
    y_pred = clf.predict(X_test)

    print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

    ## With cross validation
    # scores = cross_val_score(clf, X, Y, cv=5)
    # for v in scores:
    #     print(str(v).replace('.', ','))

0,4876289815275336
0,49354304287825823
0,6122253122526322
0,6872955384475282


In [5]:
## Privatization: Laplace privatizer
from privatizers import laplacePrivatizer

for data_set in data_sets:
    ad = adapter.GeneralAdapter(7, 1)

    dataInput = ad.fromRaw(data_set['samples'])
    dataTarget = ad.fromRaw(data_set['labels'])

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)

    inputSensitivity = privatizer.getSensitivityList(dataInput)
    targetSensitivity = 7

    epsilon = [1.0, 1.5, 2.0, 2.5, 3.0]

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)
    privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)
    for i in epsilon:
        privatizer = laplacePrivatizer.LaplacePrivatizer(i)
        privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)

        privateTargetsFloat = privatizer.privatize(dataTarget, sensitivityList = targetSensitivity)
        privateTargets = ad.toDiscreteValue(privateTargetsFloat)

        clf = svm.SVC(gamma='scale')
        clf.fit(privateData, privateTargets)

        y_pred = clf.predict(X_test)
        print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

0,4876289815275336
0,48713161895021473
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
0,4876289815275336
